In [34]:
import json
import pathlib
from dotenv import load_dotenv

import openai
from langchain.llms import OpenAI
from langchain.docstore import Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer
from langchain.agents import load_tools
from langchain.tools import StructuredTool


In [3]:

load_dotenv(dotenv_path=pathlib.Path('.env'))

True

In [51]:
llm = OpenAI(temperature=0) #gpt-3.5-turbo-16k # model_name="gpt-3.5-turbo"


In [5]:
tools = []

In [6]:
tools = load_tools(["llm-math"], llm=llm)

In [45]:

import os
from langchain.tools import tool



class Docstore:
    def __init__(self, folder_path):
        self.folder_path = folder_path

    def list_files(self, *args, **kwargs):
        """List all files in the document store recursively."""
        all_files = []
        for dirpath, dirnames, filenames in os.walk(self.folder_path):
            for filename in filenames:
                all_files.append(os.path.join(dirpath, filename))
        return all_files

    def read_file(self, filepath):
        """Read the content of a specific file."""
        filepath = filepath.replace("'", "")
        with open(filepath, 'rb') as file:
            return file.read()
        



def create_ticket(title: str, description: str, tags=[]):
    """Create a ticket or code feature request as an output JSON file. with the given input parameters"""
    output_folder = "./tickets/"
    # Create a ticket dictionary with the provided details
    ticket = {
        "title": title,
        "description": description,
        "tags": tags,
        "status": "Open"
    }
    
    # Generate a unique filename based on the title
    filename = title.replace(" ", "_") + ".json"
    filepath = os.path.join(output_folder, filename)
    
    # Write the ticket to a JSON file
    with open(filepath, 'w') as file:
        json.dump(ticket, file, indent=4)
    
    return f"Ticket saved to {filepath}"

docstore = Docstore(folder_path="./demo_app/backend/app/app/")

tools.append(
    Tool(
        name="ListFiles",
        func=docstore.list_files,
        description="Lists all txt files in the document store.",
    )
)

tools.append(
    Tool(
        name="ReadFile",
        func=docstore.read_file,
        description="Reads the content of a specific code file.",
    )
)

ticket_tool = StructuredTool.from_function(create_ticket)



tools.append(
    ticket_tool
)

In [46]:
tech_lead_agent_system_prompt: str = """
You are a tech lead, you have access to multiple repos, given a feature request, your job is to go through the code files and create a list of tickets for a software engineer to work on

"""

In [49]:
react = initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [50]:
react.run(
    f"{tech_lead_agent_system_prompt} - please create a simple ticket - DO NOT READ THE FILES"
)



> Entering new  chain...
Action:
```
{
  "action": "create_ticket",
  "action_input": {
    "title": "Simple Ticket",
    "description": "Create a simple ticket",
    "priority": "Medium",
    "tags": []
  }
}
```
Observation: Ticket saved to ./tickets/Simple_Ticket.json
Thought:The ticket has been created successfully.

> Finished chain.


'The ticket has been created successfully.'